In [ ]:
from torchvision import models
import torch
import pickle as pk # to save list as file
import gc # garbage collector, can help you free memory
!pip install import_ipynb
import import_ipynb # tool allow you to import from .ipynb files. Usualy import only from .py files.
from os.path import exists



from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import hamming_loss




# torch can use CUDA, this line check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from create_dataset import DicomDataset, DicomTestDataset # my dataset class
from create_dataset import train_metadata_df # import dataset csv

In [ ]:
# Creating instance of DicomDataset
from torchvision.transforms import ToTensor, ToPILImage, Compose
from torchvision.transforms import RandAugment # makes troubles with dataloader
from torchvision.transforms import AutoAugmentPolicy.IMAGENET, AutoAugment

rand_aug_transformation = Compose([
    ToTensor(),
    AutoAugment(AutoAugmentPolicy.IMAGENET)])

# policies = [T.AutoAugmentPolicy.CIFAR10, T.AutoAugmentPolicy.IMAGENET, T.AutoAugmentPolicy.SVHN]
# augmenters = [T.AutoAugment(policy) for policy in policies]
# imgs = [
#     [augmenter(orig_img) for _ in range(4)]
#     for augmenter in augmenters
# ]

train_dataset = DicomDataset(
    merged_df=train_metadata_df,
    image_dir="train",
#    train=True,
#    download=True,
    preload = False,
    transform=rand_aug_transformation
)



test_dataset = DicomTestDataset(
    image_dir="test",
#    train=True,
#    download=True,
    preload = False,
    transform=ToTensor()
)

test_augmented_dataset = DicomTestDataset(
    image_dir="test",
#    train=True,
#    download=True,
    preload = False,
    transform=rand_aug_transformation
)

# >>> transforms.Compose([
# >>>     transforms.CenterCrop(10),
# >>>     transforms.PILToTensor(),
# >>>     transforms.ConvertImageDtype(torch.float),
# >>> ])
#augmenter = T.RandAugment()

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

train_dataloader.__iter__()

In [ ]:
for list_of_tensor_images,list_of_labels in train_dataloader:
    print(list_of_tensor_images.shape,"LABELS !!!!",list_of_labels)
    break


In [ ]:
model = models.alexnet(pretrained=True)


torch.cuda.empty_cache()
#model = model_dict[model_name]
model.eval()
model.to(device)

ds_res= []
#print("start ",model_name)
#for image,lable in data_set:
for list_of_tensor_images,list_of_labels in (train_dataloader):
    #print(image)
    with torch.no_grad():
        features = model(list_of_tensor_images.to(device))
        #features = model(torch.unsqueeze(image,0).to(device))
        print(type(features),type(features.numpy()),features.numpy().shape,len([ elem.numpy() for elem in features ]),features.shape)

    
    #alex_np_fea = [ elem.numpy()[0] for elem in alex_features ]
    sk_feat = [ elem for elem in features.numpy() ]
    print(len(sk_feat),len(sk_feat[0]),sk_feat)
    print(len(list_of_labels),len(list_of_labels[0]),list_of_labels)
    rfc = RandomForestClassifier(n_jobs=7, n_estimators= 100)
    rfc.fit(sk_feat,list_of_labels)
        
    #ds_res.append(im_res.to('cpu'))
    del fetures
    torch.cuda.empty_cache()
    gc.collect()